In [1]:
import train_embeddings
import matplotlib.pyplot as plt
import sentence_features
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc, precision_recall_curve, average_precision_score
import pickle

import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D, Embedding
from keras.layers.merge import Concatenate
from keras.preprocessing import sequence

np.random.seed(1)

/Users/scott/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Hyperparameters
embedding_dim = 250
filter_sizes = (3, 5, 7)
num_filters = 64
dropout_prob = (0.3, 0.4)
hidden_dims = 64

# Training parameters
batch_size = 128
num_epochs = 8

# Prepossessing parameters
sequence_length = 200

In [3]:
# Load Data
fastText_path = "../data/models/fastText/fastText_sw_cbow.bin"
ft = train_embeddings.load_fastText_model(fastText_path)

mapped_reports_path = "../data/processed/processed_reports/preprocessed_findings_replace_sw"
mapped_reports = pickle.load(open(mapped_reports_path, 'rb'))

pipeline = make_pipeline(train_embeddings.FastTextReportVectorizer(ft, granularity="word", pad_len=200), sentence_features.LabelSeparator(), None)
data, labels = pipeline.transform(mapped_reports)

split_point = int(0.85 * len(data))

trainingX = np.array(data[:split_point])
trainingY = np.array(labels[:split_point])
print(np.unique(trainingY, return_counts=True))

testingX = np.array(data[split_point:])
testingY = np.array(labels[split_point:])
print(np.unique(testingY, return_counts=True))

(array([0, 1]), array([29145, 17266]))
(array([0, 1]), array([5118, 3073]))


In [4]:
print("Training X shape: " + str(trainingX.shape))
print("Training Y shape: " + str(trainingY.shape))

Training X shape: (46411, 200, 250)
Training Y shape: (46411,)


In [ ]:
input_shape = (sequence_length, embedding_dim)

model_input = Input(shape=input_shape)

z = Dropout(dropout_prob[0])(model_input)

conv_blocks = []
for sz in filter_sizes:
    conv = Convolution1D(filters=num_filters, kernel_size=sz, padding="valid", activation="relu", strides=1)(z)
    conv = MaxPooling1D(pool_size=2)(conv)
    conv = Flatten()(conv)
    conv_blocks.append(conv)
z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

z = Dropout(dropout_prob[1])(z)
z = Dense(hidden_dims, activation="relu")(z)
model_output = Dense(1, activation="sigmoid")(z)

model = Model(model_input, model_output)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [ ]:
model.fit(trainingX, trainingY, batch_size=batch_size, epochs=num_epochs, validation_data=(testingX, testingY), verbose=1)

Train on 46411 samples, validate on 8191 samples
Epoch 1/8
46411/46411 [==============================] - 603s - loss: 0.5451 - acc: 0.7422 - val_loss: 0.4353 - val_acc: 0.7961
Epoch 2/8
46411/46411 [==============================] - 562s - loss: 0.4111 - acc: 0.8059 - val_loss: 0.4291 - val_acc: 0.7982
Epoch 3/8
46411/46411 [==============================] - 579s - loss: 0.3620 - acc: 0.8319 - val_loss: 0.4006 - val_acc: 0.8153
Epoch 4/8
46411/46411 [==============================] - 576s - loss: 0.3173 - acc: 0.8575 - val_loss: 0.4004 - val_acc: 0.8208
Epoch 5/8
46411/46411 [==============================] - 609s - loss: 0.2749 - acc: 0.8816 - val_loss: 0.3917 - val_acc: 0.8231
Epoch 6/8
46411/46411 [==============================] - 562s - loss: 0.2446 - acc: 0.8949 - val_loss: 0.4061 - val_acc: 0.8241
Epoch 7/8
 1920/46411 [>.............................] - ETA: 467s - loss: 0.1952 - acc: 0.9219

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/scott/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-b19b8f12edf0>", line 1, in <module>
    model.fit(trainingX, trainingY, batch_size=batch_size, epochs=num_epochs, validation_data=(testingX, testingY), verbose=1)
  File "/Users/scott/anaconda3/lib/python3.6/site-packages/keras/engine/training.py", line 1598, in fit
    validation_steps=validation_steps)
  File "/Users/scott/anaconda3/lib/python3.6/site-packages/keras/engine/training.py", line 1183, in _fit_loop
    outs = f(ins_batch)
  File "/Users/scott/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py", line 2273, in __call__
    **self.session_kwargs)
  File "/Users/scott/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 895, in run
    run_metadata_ptr)
  File "/Users/scott/anaconda3/lib/python3.6/s

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/Users/scott/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/Users/scott/anaconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/Users/scott/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/Users/scott/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "/Users/scott/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/Users/scott/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 637, in _abort_queues
    self._abort_queue(stream)
  File "/Users/scott/anaconda3/lib/

In [ ]:
predictions = model.predict(testingX)

In [ ]:
fpr, tpr, thresholds = roc_curve(testingY, predictions, pos_label=1)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()